In [13]:
import requests
import pandas as pd
import numpy as np
import urllib.request

In [14]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):

    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r *1000

In [15]:
def location_splitter(location):
    location=location.split(' ')
    return location[0],location[1],location[2],location[3]

In [16]:
traffic_image_url='http://datamall2.mytransport.sg/ltaodataservice/Traffic-Imagesv2'
headers_val={'AccountKey':'AO4qMbK3S7CWKSlplQZqlA=='}
traffic_image_req=requests.get(url=traffic_image_url,headers=headers_val)
traffic_image_df=pd.DataFrame(eval(traffic_image_req.content)['value'])

In [17]:
traffic_speed_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficSpeedBandsv2'
traffic_speed_req=requests.get(url=traffic_speed_url,headers=headers_val)
traffic_speed_df=pd.DataFrame(eval(traffic_speed_req.content)['value'])
road_category_letter=['A','B','C','D','E','F','G']
road_category_type=['Expressways','Major Arterial Roads','Arterial Roads','Minor Arterial Roads','Small Roads','Slip Roads','No category info available']
category_mapping=pd.DataFrame({'RoadCategory':road_category_letter,'category':road_category_type})
traffic_speed_df=traffic_speed_df.merge(category_mapping,'left','RoadCategory')
traffic_speed_df['start_latitude'],traffic_speed_df['start_longitude'],traffic_speed_df['end_latitude'],traffic_speed_df['end_longitude']=np.vectorize(location_splitter)(traffic_speed_df['Location'])
traffic_speed_df=traffic_speed_df.drop(['LinkID','RoadCategory','SpeedBand','MaximumSpeed','Location'],axis=1)
traffic_speed_df=traffic_speed_df.astype({'start_latitude':float,'start_longitude':float,'end_latitude':float,'end_longitude':float})

In [18]:
traffic_incidents_url='http://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents'
traffic_incidents_req=requests.get(url=traffic_incidents_url,headers=headers_val)
traffic_incidents_df=pd.DataFrame(eval(traffic_incidents_req.content)['value'])


# NEA API
For some reason this one need to constantly call and keep track of previous results. Sometimes calls does not return all data.
Some names are given as S123, etc, looking to try OneMap reverse geocoding to get road name from lat long.

In [19]:
weatherreq=requests.get(url='https://api.data.gov.sg/v1/environment/rainfall')
weather_df=pd.DataFrame(eval(weatherreq.content)['metadata']['stations'])

weather_df['latitude']=weather_df['location'].apply(lambda x: x['latitude'])
weather_df['longitude']=weather_df['location'].apply(lambda x: x['longitude'])
weather_df['timestamp']=eval(weatherreq.content)['items'][0]['timestamp']
weather_df['timestamp']=pd.to_datetime(weather_df['timestamp'])

station_rainfall=pd.DataFrame(eval(weatherreq.content)['items'][0]['readings']).rename(columns={'value':'rainfall'})

weather_df=weather_df.merge(station_rainfall,how='left',left_on='id',right_on='station_id')
weather_df=weather_df.drop(['id','device_id','station_id','location'],axis=1)

# Onemap

In [20]:
onemapTokenAPIResponse=requests.post('https://developers.onemap.sg/privateapi/auth/post/getToken',json={'email':'leejin@u.nus.edu','password':'Whysohardtochange123!'})#.content
onemapAPItoken=eval(onemapTokenAPIResponse.content)['access_token']

In [21]:
def roadnamegrabber(latitude,longitude):
    tempResponse=requests.get('https://developers.onemap.sg/privateapi/commonsvc/revgeocode?location=%s,%s&token=%s&buffer=100&addressType=all'%(str(latitude),str(longitude),onemapAPItoken))
    name=pd.DataFrame(eval(tempResponse.content)['GeocodeInfo'])['ROAD'].mode()[0]
    return name.title()

# Calculations


In [22]:
traffic_image_df['key']=0
traffic_incidents_df['key']=0
weather_df['key']=0

In [23]:
nearest_incidents=traffic_image_df.merge(traffic_incidents_df,'outer','key')
nearest_incidents['incident_distance_from_id']=(np.vectorize(haversine)(nearest_incidents['Latitude_x'],nearest_incidents['Longitude_x'],nearest_incidents['Latitude_y'],nearest_incidents['Longitude_y']))
nearest_incidents=nearest_incidents[nearest_incidents['incident_distance_from_id']<500].sort_values('incident_distance_from_id')
nearest_incidents=nearest_incidents[['CameraID','Type','Message']]

In [24]:
nearest_weather=traffic_image_df.merge(weather_df,'outer','key')
nearest_weather['distance_from_id']=(np.vectorize(haversine)(nearest_weather['Latitude'],nearest_weather['Longitude'],nearest_weather['latitude'],nearest_weather['longitude']))
nearest_weather=nearest_weather.sort_values('distance_from_id').groupby('CameraID').head(1).sort_values('distance_from_id')
nearest_weather=nearest_weather[['CameraID','Latitude','Longitude','ImageLink','rainfall']]
nearest_weather['name']=np.vectorize(roadnamegrabber)(nearest_weather['Latitude'],nearest_weather['Longitude'])

In [25]:
final_df=nearest_weather.merge(nearest_incidents,'left','CameraID')
final_df

,CameraID,Latitude,Longitude,ImageLink,rainfall,name,Type,Message
0,1701,1.323605,103.858780,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Saint George'S Lane,NaN,NaN
1,3795,1.301451,103.910596,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Still Road South,NaN,NaN
2,4706,1.297920,103.782050,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Lower Kent Ridge Road,NaN,NaN
3,2702,1.445554,103.768340,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Woodlands Crossing,NaN,NaN
4,6713,1.340298,103.945652,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Tampines Street 92,NaN,NaN
...,...,...,...,...,...,...,...,...
122,8702,1.390590,103.771700,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Segar Road,NaN,NaN
123,1503,1.270664,103.856978,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Marina Coastal Drive,NaN,NaN
124,2708,1.386500,103.774700,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Bukit Timah Expressway,NaN,NaN
125,2703,1.350478,103.791034,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Rifle Range Road,NaN,NaN


In [27]:
final_df.to_csv('main_df.csv',index=False)

In [28]:
final_df[final_df['Message'].notna()]

,CameraID,Latitude,Longitude,ImageLink,rainfall,name,Type,Message
8,4714,1.310230,103.764380,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Clementi Avenue 3,Roadwork,(28/10)0:27 Roadworks on Clementi Avenue 6 (to...
11,5797,1.322875,103.891079,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Jalan Afifi,Roadwork,(28/10)0:01 Roadworks on KPE (towards ECP) bef...
14,4704,1.278770,103.823750,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Ayer Rajah Expressway,Roadwork,(27/10)20:59 Roadworks on Telok Blangah Road (...
15,4704,1.278770,103.823750,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Ayer Rajah Expressway,Roadwork,(28/10)0:39 Roadworks in Sentosa Gateway Tunne...
17,4709,1.312019,103.763002,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Clementi Avenue 3,Roadwork,(28/10)0:27 Roadworks on Clementi Avenue 6 (to...
...,...,...,...,...,...,...,...,...
107,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Geylang Bahru,Roadwork,(28/10)0:02 Roadworks on KPE (towards TPE) aft...
108,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Geylang Bahru,Roadwork,(27/10)22:33 Roadworks on Sims Avenue (toward ...
109,1004,1.319536,103.875067,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Geylang Bahru,Roadwork,(28/10)0:08 Roadworks on Macpherson Road (towa...
111,8706,1.366400,103.708990,https://dm-traffic-camera-itsc.s3.ap-southeast...,0.0,Kranji Expressway,Roadwork,(27/10)22:16 Roadworks on KJE (towards PIE) at...


In [58]:
final_df.loc[final_df['CameraID']=='1004',][['Message']].itertuples()

In [77]:
incident_res=''
new_line = '\n'
for i in final_df.loc[final_df['CameraID']=='1004',][['Message']].reset_index(drop=True).itertuples():
    incident_res+= '%s. %s \n'%(str(i[0]+1),i[1])
incident_res

'1. (28/10)0:02 Roadworks on KPE (towards TPE) after Nicoll Highway Entrance. Avoid lane 1. \n2. (27/10)22:33 Roadworks on Sims Avenue (toward Changi) after KPE. Avoid left lane. \n3. (28/10)0:08 Roadworks on Macpherson Road (towards Genting Lane) after Lorong Bakar Batu. \n'

In [71]:
new_line = '\n'
print(f"Nearby Incidents:{new_line}{new_line.join(incident_res)}")

Nearby Incidents:
1. (28/10)0:02 Roadworks on KPE (towards TPE) after Nicoll Highway Entrance. Avoid lane 1.
2. (27/10)22:33 Roadworks on Sims Avenue (toward Changi) after KPE. Avoid left lane.
3. (28/10)0:08 Roadworks on Macpherson Road (towards Genting Lane) after Lorong Bakar Batu.


In [66]:
f"{new_line}{new_line.join(incident_res)}"

'\n1. (28/10)0:02 Roadworks on KPE (towards TPE) after Nicoll Highway Entrance. Avoid lane 1.\n2. (27/10)22:33 Roadworks on Sims Avenue (toward Changi) after KPE. Avoid left lane.\n3. (28/10)0:08 Roadworks on Macpherson Road (towards Genting Lane) after Lorong Bakar Batu.'

In [ ]:
# Downloads all the csv files
# traffic_image_df.to_csv('traffic_image.csv',index=False)
# traffic_speed_df.to_csv('traffic_speed.csv',index=False)
# traffic_incidents_df.to_csv('traffic_incidents.csv',index=False)
# weather_df.to_csv('weather.csv',index=False)